Merge draft datasets

In [152]:
import pandas as pd

draftnet_f = './data/nbadraft_strengths_weaknesses.csv'
alltimedraft_f = '../../nba_analysis_app/data/alltime_draft.csv'

draftnet_df = pd.read_csv(draftnet_f)
alltimedraft_df = pd.read_csv(alltimedraft_f)

# for getting a column to join on
alltimedraft_df['player'] = alltimedraft_df.PLAYER_NAME.str.lower().str.replace(' ','-')

In [153]:
alltimedraft_df['player'] = alltimedraft_df.PLAYER_NAME.str.lower().str.replace(' ','-')

In [154]:
keep_cols = ['PERSON_ID','PLAYER_NAME','SEASON','draft_year','strengths','weaknesses']
rename_dict = {'PERSON_ID':'person_id','PLAYER_NAME':'player','SEASON':'season'}
df = pd.merge(alltimedraft_df, draftnet_df, on='player', how='inner')[keep_cols]
df.rename(rename_dict, axis=1, inplace=True)
df = df.loc[df.season >= 2006]
df.head()

,person_id,player,season,draft_year,strengths,weaknesses
0,1631094,Paolo Banchero,2022,2022,"Has good size and length for his position, sta...","For all of his offensive gifts, still has room..."
1,1631096,Chet Holmgren,2022,2022,Extremely skilled frontcourt player who can in...,Lack of physical strength (195 lbs) remains hi...
2,1631099,Keegan Murray,2022,2022,"A 6’8 225 frontcourt player with the frame, le...",Will have some questions about his true positi...
3,1631093,Jaden Ivey,2022,2022,"An aggressive explosive, 6’4 200 lb combo guar...",Ivey’s hard charging energy and play style can...
4,1631097,Bennedict Mathurin,2022,2022,"6’7 wing with tremendous size, maturity, explo...",The biggest hurdle for Mathurin at this point ...


Add in the all star information

In [155]:
allstar_f = './data/nba_allstar_all.csv'
allstar_df = pd.read_csv(allstar_f)
allstar_df = allstar_df.loc[allstar_df.year >= 2001]

In [156]:
import numpy as np
all_star_indicator = []
all_star_first_year = []
for index, row in df.iterrows():
    if row['player'] in allstar_df['Player'].values:
        all_star_indicator.append(1)
        player_allstar = allstar_df.loc[allstar_df['Player'] == row['player']]
        year_min = player_allstar.year.min()
        allstar_player_year = year_min - row['season']
        all_star_first_year.append(allstar_player_year)
    else:
        all_star_indicator.append(0)
        all_star_first_year.append(np.nan)
df['allstar_bool'] = all_star_indicator
df['allstar_first_year'] = all_star_first_year
df['within7'] = (df.allstar_first_year <= 7).astype(int)
df['within5'] = (df.allstar_first_year <= 5).astype(int)
df.drop_duplicates(subset='player', inplace=True)

nlp feature extraction

within 5 years

In [157]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [158]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing Punctuation
    tokens = [token for token in tokens if token not in string.punctuation]
    
    # Removing Stop Words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Joining tokens back into a single text
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [159]:
all_df = pd.concat([df, draftnet_df])
all_df['text'] = all_df['strengths'] + ' ' + all_df['weaknesses']
all_df['text'] = all_df['text'].apply(preprocess_text)

vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the max_features parameter
X = vectorizer.fit_transform(all_df['text'])
# inds = np.where(all_df['draft_year'] == '2023')[0]
# X_2023 = X[inds]
# X = X[all_df['draft_year'] != '2023']

In [176]:
n = 7
if n == 5:
    mask = (all_df.within5 == 1) | (all_df.season <= 2018)
    # data = df.loc[(df.within5 == 1) | (df.season <= 2018)]
else:
    mask = (all_df.within7 == 1) | (all_df.season <= 2016)
    # data = df.loc[(df.within7 == 1) | (df.season <= 2016)]

mask = mask & (all_df.season != 2015)
# data['text'] = data['strengths'] + ' ' + data['weaknesses']
# data['text'] = data['text'].apply(preprocess_text)

# vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the max_features parameter
# X = vectorizer.fit_transform(data['text'])
X_rest = X[mask]
y = all_df[mask]['within7']

In [177]:
# Step 4: Model Training
X_train, X_test, y_train, y_test = train_test_split(X_rest, y, test_size=0.2, random_state=42, stratify=y)
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [178]:
from sklearn.metrics import accuracy_score

# Predict on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print('Model Accuracy:', accuracy)

Model Accuracy: 0.8461538461538461


In [179]:
from sklearn.model_selection import KFold, cross_validate
kf = KFold(n_splits=6, shuffle=True, random_state=2)
scoring = ['accuracy','precision','recall','f1','roc_auc','neg_log_loss']
model = LogisticRegression()
scores = cross_validate(model, X_train, y_train, cv=kf, scoring=scoring)

for metric in scoring:
    metric_scores = scores[f'test_{metric}']
    print(f'{metric}: {metric_scores}')

accuracy: [0.83333333 0.77777778 0.88235294 0.82352941 0.82352941 0.94117647]
precision: [0. 0. 0. 0. 0. 0.]
recall: [0. 0. 0. 0. 0. 0.]
f1: [0. 0. 0. 0. 0. 0.]
roc_auc: [0.86666667 0.96428571 0.83333333 0.45238095 0.66666667 0.9375    ]
neg_log_loss: [-0.43724088 -0.5329768  -0.35886836 -0.47004793 -0.46387763 -0.27259668]


c:\Users\Andy\anaconda3\envs\nba-stats\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Andy\anaconda3\envs\nba-stats\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Andy\anaconda3\envs\nba-stats\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Andy\anaconda3\envs\nba-stats\lib\site-packages\sklearn\metrics\_cl

In [184]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV

from sklearn.utils import class_weight

# Calculate class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

param_grid = {
    'C': [5, 10, 15, 20],
    'penalty': ['l1','l2'],
    'solver': ['liblinear'],
    'class_weight': [None, class_weights]
}

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model = LogisticRegression()
grid_search = GridSearchCV(model, param_grid, cv=kf, scoring='accuracy')

grid_search.fit(X_train, y_train)

print('Best hyperparameters: ', grid_search.best_params_)
print('Best Score: ', grid_search.best_score_)

Best hyperparameters:  {'C': 5, 'class_weight': None, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score:  0.846190476190476


c:\Users\Andy\anaconda3\envs\nba-stats\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Andy\anaconda3\envs\nba-stats\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Andy\anaconda3\envs\nba-stats\lib\site-packages\sklearn\linear_model\_logistic.py", line 1160, in fit
    self._validate_params()
  File "c:\Users\Andy\anaconda3\envs\nba-stats\lib\site-packages\sklearn\base.py", line 600, in _validate_params
    validate_parameter

Now try it out on this year's draft class

In [181]:
# data = draftnet_df.loc[draftnet_df.draft_year == '2023']
# data['text'] = data['strengths'] + ' ' + data['weaknesses']
# data['text'] = data['text'].apply(preprocess_text)

# mask = all_df['draft_year'] == '2023'
# inds = np.where(all_df['draft_year'] == '2023')[0]
# X_2023 = X[mask]

# vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the max_features parameter
# tmp_X = vectorizer.fit_transform(data['text'])
# tmp_X = 

model = LogisticRegression(**grid_search.best_params_)
model.fit(X_rest, y)
# y_pred = model.predict(data['text'])

LogisticRegression(C=5, penalty='l1', solver='liblinear')

In [182]:
mask = all_df['season'] == 2015
# mask = all_df['draft_year'] == '2023'
X_2023 = X[mask]
y_pred_prob = model.predict_proba(X_2023)

In [183]:
df_2023 = all_df[mask]
df_2023['allstar_prob'] = [x[1] * 100 for x in y_pred_prob]
df_2023 = df_2023[['player','allstar_prob']]
df_2023.head()

C:\Users\Andy\AppData\Local\Temp\ipykernel_9980\443112442.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['allstar_prob'] = [x[1] * 100 for x in y_pred_prob]


,player,allstar_prob
256,Karl-Anthony Towns,9.598615
257,Jahlil Okafor,12.225538
258,Kristaps Porzingis,29.367995
259,Stanley Johnson,10.481706
260,Frank Kaminsky,18.665252
